In [177]:
import numpy as np
import pandas as pd
from collections import  Counter

In [178]:
credit = pd.read_csv('../data/Xtrain.csv')  #for trian
# credit = pd.read_csv('../data/Xtest.csv')     # for predict
print(credit.shape)
credit =credit.drop_duplicates()
print(credit.shape)

(50000, 24)
(50000, 24)


In [179]:
train_y = pd.read_csv('../data/Ytrain.csv')
train_y.shape


(50000, 2)

In [180]:
Counter(train_y.ChargeOff)
## 样本均衡

Counter({0: 25152, 1: 24848})

In [181]:
credit.isnull().sum()

Id                     0
Name                   1
City                   0
State                  0
Zip                    0
Bank                  76
BankState             77
NAICS                  0
ApprovalDate           0
ApprovalFY             0
Term                   0
NoEmp                  0
NewExist               7
CreateJob              0
RetainedJob            0
FranchiseCode          0
UrbanRural             0
RevLineCr            153
LowDoc               172
DisbursementDate     119
DisbursementGross      0
BalanceGross           0
GrAppv                 0
SBA_Appv               0
dtype: int64

In [182]:
# 在输入数据中加入label 方便操作
credit.insert(24,'MIS_Status',train_y.ChargeOff)

In [183]:
# 将金额转为float
currency = [20,21,22,23]
for i in currency:
    credit[credit.columns[i]] = credit[credit.columns[i]].replace('[\$,]', '', regex=True).astype(float) 

In [184]:
## 对于State 和 BankState 进行one-hot 编码
credit = pd.concat([credit,pd.get_dummies(credit.State)],axis=1)
temp = pd.get_dummies(credit.BankState)
new_keys = ['B_'+key for key in temp.keys()]
temp.columns = new_keys
credit = pd.concat([credit,temp],axis=1)
credit.head()

,Id,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,...,B_SD,B_TN,B_TX,B_UT,B_VA,B_VT,B_WA,B_WI,B_WV,B_WY
0,0,"NPE Realty, LLC",Pembroke Pines,FL,33027,"TD BANK, NATIONAL ASSOCIATION",DE,334516,16-Apr-10,2010,...,0,0,0,0,0,0,0,0,0,0
1,1,"KEVCO CONSTRUCTION, LLC",SANDY,UT,84094,ZIONS FIRST NATIONAL BANK,UT,236115,30-Jan-10,2010,...,0,0,0,1,0,0,0,0,0,0
2,2,EAST L A SHOE REPAIR,LOS ANGELES,CA,90022,BANK OF AMERICA NATL ASSOC,NC,811430,29-Jul-02,2002,...,0,0,0,0,0,0,0,0,0,0
3,3,"RUSSLER, DANIEL C, MD",LODI,WI,53555,ASSOCIATED BANK NATL ASSOC,WI,0,10-Jun-94,1994,...,0,0,0,0,0,0,0,1,0,0
4,4,AFFORDABLE FAMILY DENISTRY LLC,LEBANON,MO,65536,COMMERCE BANK,MO,621210,6-Feb-06,2006,...,0,0,0,0,0,0,0,0,0,0


# 删除一部分特征

In [185]:
credit = credit.drop(axis =1, columns = ['Id','City','Name','Bank', 'BankState', 'DisbursementDate', 'RevLineCr','State','NAICS'])
credit = credit.drop(axis =1, columns = ['Zip','ApprovalDate','ApprovalFY','BalanceGross','SBA_Appv'])


In [186]:
credit.head()

,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,LowDoc,DisbursementGross,GrAppv,...,B_SD,B_TN,B_TX,B_UT,B_VA,B_VT,B_WA,B_WI,B_WV,B_WY
0,130,25,1.0,2,0,0,1,N,2000000.0,2000000.0,...,0,0,0,0,0,0,0,0,0,0
1,33,6,1.0,1,3,0,1,N,17000.0,17000.0,...,0,0,0,1,0,0,0,0,0,0
2,60,1,1.0,1,1,1,1,N,17000.0,17000.0,...,0,0,0,0,0,0,0,0,0,0
3,160,1,2.0,0,0,1,0,N,137000.0,137000.0,...,0,0,0,0,0,0,0,1,0,0
4,101,2,1.0,7,2,1,1,N,46665.0,25000.0,...,0,0,0,0,0,0,0,0,0,0


# 处理NewExist 异常值

In [191]:
Counter(credit.NewExist)

Counter({1.0: 35636,
         2.0: 14328,
         0.0: 29,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1})

In [192]:
#We have a feature 'RetainedJob' - From the document, it shows number of jobs retained. 
#I can assume that if loan retains some jobs it is an existing business.
# I will create a condition and iterate through loop to 
#assign new value "1" which is Existing business to those rows where Retained Job is >= 1
cond_2 = credit[(credit['NewExist'] == 0) & (credit['RetainedJob'] >=1)].index
for i in cond_2:
    credit.loc[i,['NewExist']] = 1 

In [193]:
#Next we will do the same thing with another condition: isnull and Retained Job >=1
cond_3 = credit[(credit.NewExist.isnull()) & (credit['RetainedJob'] >=1)].index
for i in cond_3:
    credit.loc[i,['NewExist']] = 1    
    
credit = credit.dropna(axis =0, subset=['NewExist'])   ##for train
# credit.NewExist = credit.NewExist.fillna(1)   ## for test

# 处理LowDoc 异常值  

In [197]:
# LowDoc 是一种立刻能得到15万刀的短期贷款
# 这里简单的删去 异常条目
Counter(credit.LowDoc)

Counter({'N': 44713, 'Y': 4934})

In [198]:
cond_6 = credit[(credit['LowDoc'] != "Y") & (credit['LowDoc'] != "N")]
for i in cond_6.index:
    credit.loc[i,'LowDoc'] = np.nan

credit = credit.dropna(axis =0, subset=['LowDoc'])  ## for train
# credit.LowDoc = credit.LowDoc.fillna('N')   #for test

In [199]:
def yes_no(i):
    if i > 0:
        return 1
    return 0

credit.Term = credit.Term//12
credit['IscreateJob'] = credit.CreateJob.apply(yes_no)
credit['IsRetained'] = credit.RetainedJob.apply(yes_no)
credit['IsFranchise'] = credit.FranchiseCode.apply(yes_no)

def lowdoc(i):
    if i == "Y":
        return 1
    return 0
credit.LowDoc = credit.LowDoc.apply(lowdoc)
credit = credit.drop(axis =1, columns = ['FranchiseCode'])


In [200]:
#Split data into train and test sets + label target value
from sklearn.model_selection import train_test_split
y = credit.MIS_Status
X = credit.drop(['MIS_Status'], axis=1)
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.25, random_state=0)

In [201]:
credit.head()

,Term,NoEmp,NewExist,CreateJob,RetainedJob,UrbanRural,LowDoc,DisbursementGross,GrAppv,MIS_Status,...,B_UT,B_VA,B_VT,B_WA,B_WI,B_WV,B_WY,IscreateJob,IsRetained,IsFranchise
0,10,25,1.0,2,0,1,0,2000000.0,2000000.0,0,...,0,0,0,0,0,0,0,1,0,0
1,2,6,1.0,1,3,1,0,17000.0,17000.0,1,...,1,0,0,0,0,0,0,1,1,0
2,5,1,1.0,1,1,1,0,17000.0,17000.0,0,...,0,0,0,0,0,0,0,1,1,1
3,13,1,2.0,0,0,0,0,137000.0,137000.0,0,...,0,0,0,0,1,0,0,0,0,1
4,8,2,1.0,7,2,1,0,46665.0,25000.0,0,...,0,0,0,0,0,0,0,1,1,1


In [208]:

import xgboost as xgb
# os.environ['KMP_DUPLICATE_LIB_OK']='True'
#Train the XGboost Model for Classification
#Model with default parameters
model = xgb.XGBClassifier()
train_model = model.fit(train_X, train_y)

In [211]:
pred = train_model.predict(test_X)

In [212]:
from sklearn.metrics import accuracy_score
print("Accuracy for model: %.2f" % (accuracy_score(test_y, pred) * 100))

Accuracy for model: 87.33


In [213]:
n_folds = 5
early_stopping = 20
params = {'eta': 0.3, 'max_depth': 5, 'subsample': 0.7, 'colsample_bytree': 0.7, 'objective': 'binary:logistic', 'seed': 99, 'silent': 1, 'eval_metric':'auc', 'nthread':4}

In [214]:
## cross validation
xg_train = xgb.DMatrix(train_X, label=train_y);
cv = xgb.cv(params, xg_train, 5000, nfold=n_folds,early_stopping_rounds=early_stopping, verbose_eval=1)

[0]	train-auc:0.74512+0.00193	test-auc:0.74238+0.00546
[1]	train-auc:0.84357+0.07298	test-auc:0.84006+0.07183
[2]	train-auc:0.87562+0.05936	test-auc:0.87210+0.05742
[3]	train-auc:0.91153+0.01298	test-auc:0.90871+0.01230
[4]	train-auc:0.91830+0.00400	test-auc:0.91488+0.00487
[5]	train-auc:0.92165+0.00246	test-auc:0.91811+0.00385
[6]	train-auc:0.92342+0.00201	test-auc:0.91955+0.00384
[7]	train-auc:0.92639+0.00154	test-auc:0.92259+0.00265
[8]	train-auc:0.92770+0.00153	test-auc:0.92398+0.00318
[9]	train-auc:0.92943+0.00147	test-auc:0.92539+0.00305
[10]	train-auc:0.93105+0.00178	test-auc:0.92676+0.00331
[11]	train-auc:0.93206+0.00172	test-auc:0.92734+0.00325
[12]	train-auc:0.93368+0.00130	test-auc:0.92888+0.00315
[13]	train-auc:0.93497+0.00128	test-auc:0.93002+0.00339
[14]	train-auc:0.93606+0.00115	test-auc:0.93078+0.00306
[15]	train-auc:0.93715+0.00102	test-auc:0.93176+0.00269
[16]	train-auc:0.93774+0.00108	test-auc:0.93202+0.00249
[17]	train-auc:0.93849+0.00110	test-auc:0.93252+0.00227
[1

In [176]:
clf = xgb.train(params,xg_train,num_boost_round=cv.shape[0])

In [168]:
dtest = xgb.DMatrix(credit)
pred = clf.predict(dtest)

In [169]:
pred = [1 if p>= 0.5 else 0 for p in pred]

In [172]:
pred = pd.DataFrame(pred)
pred.columns = ['ChargeOff']
ids = [i for i in range(100000)]
ids = pd.DataFrame(ids)
ids.columns = ['Id']
res = pd.concat([ids,pred],axis=1)


In [175]:
res.to_csv('../res/output.csv',index=False)